In [46]:
# Load libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from folktables import ACSDataSource, ACSPublicCoverage 

import pickle
import os
import os
if not os.path.exists('./dataset'):
    os.makedirs('./dataset')


STATE_LIST = ['VA']


In [47]:
ACS_categories = {
            "ANC": {
                1.0: "Single",
                2.0: "Multiple",
                3.0: "Unclassified",
                4.0: "Not reported",
            },
    
            "DIS": {
                1.0: "Yes",
                2.0: "No",
            },
            "CIT": {
                1.0: "Born in the United States",
                2.0: "Born in Puerto Rico, Guam, the U.S. Virgin Islands, or Northern Marianas",
                3.0: "Born abroad of U.S. citizen parent or parents",
                4.0: "U.S. citizen by naturalization",
                5.0: "Not a U.S. citizen",
            },
            "DEAR": {
                1.0: "Yes",
                2.0: "No",
            },
            "DEYE": {
                1.0: "Yes",
                2.0: "No",
            },
            "DREM": {
                1.0: "Yes",
                2.0: "No",
            },
            "ESR": {
                1.0: "Civilian employed, at work",
                2.0: "Civilian employed, with a job but not at work",
                3.0: "Unemployed",
                4.0: "Armed forces, at work",
                5.0: "Armed forces, with a job but not at work",
                6.0: "Not in labor force"
            },
            "MAR": {
                1.0: "Married",
                2.0: "Widowed",
                3.0: "Divorced",
                4.0: "Separated",
                5.0: "Never married or under 15 years old",
            },
            "MIG": {
                1.0: "Lived in same house a year ago",
                2.0: "Lived outside US and Puerto Rico",
                3.0: "Lived in a different house in US or Puerto Rico",
            },
            "MIL": {
                1.0: "Now on active duty",
                2.0: "On active duty in the past, but not now",
                3.0: "Only on active duty for training in Reserves/National Guard",
                4.0: "Never served in the military",
            },
            "NATIVITY": {
                1.0: "Native",
                2.0: "Foreign born",
            },
            
    
            "SEX": {1.0: "Male", 2.0: "Female"},
            "RAC1P": {
                1.0: "White alone",
                2.0: "Black or African American alone",
                3.0: "American Indian alone",
                4.0: "Alaska Native alone",
                5.0: (
                    "American Indian and Alaska Native tribes specified;"
                    "or American Indian or Alaska Native,"
                    "not specified and no other"
                ),
                6.0: "Asian alone",
                7.0: "Native Hawaiian and Other Pacific Islander alone",
                8.0: "Some Other Race alone",
                9.0: "Two or More Races",
            },
        }
for STATE in STATE_LIST:
    data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
    ca_data = data_source.get_data(states=[STATE], download=True)

    df, labels, _ = ACSPublicCoverage.df_to_pandas(ca_data, categories=ACS_categories, dummies=False)
    df.columns

    df = df.drop(columns=['ESP','FER'])
    df = df.dropna(subset=['MIL', 'ESR'])

    renames = {'AGEP':'Age',
               'ANC':'Ancestry recode',
               'CIT':'Citizenship',
               'DIS':'Disability',
               'DEAR': 'Hearing Difficulty',
               'DEYE': 'Vision Difficulty',
               'DREM': 'Cognitive Difficulty',
               'ESR': 'Employment Status',
               'MIG':'Mobility status',
               'MIL':'Military Service',
               'NATIVITY':'Nativity',
               'SCHL':'Education',
               'RAC1P': 'Race',
               'PINCP': 'Income',
               'MAR':'Marriage',
               'SEX':'Sex'} 

    df = df.rename(columns = renames)
    df['Label']= labels
    df['Label'] = df['Label'].replace({True: 1.0, False: 0.0})
    with open(f'./dataset/ACS_PublicCoverage_{STATE}.pickle', 'wb') as f:
        pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)

/tmp/ipykernel_839119/225645514.py:110: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Label'] = df['Label'].replace({True: 1.0, False: 0.0})
